## 講習2 --- IRAFで1次処理

IRAFのタスクを使って、**１次処理**(生データからダークを引き、それをフラットで割り、スカイバイアスを引く) を行ってみましょう。  

ここでは、IRAFの基本タスク、imarith, imcombineを使います。  

通常、近赤外線観測ではそのあとにregistration(複数枚のイメージの位置合わせ)をしてスタックするという作業までして1次処理なのですが、ここでは時間の関係上省略します。(appendixとして資料を追加します。)  

### サンプルデータ

sample_data2　のディレクトリの中には、サンプルの生データセットがあります。  
南アフリカIRSF望遠鏡のSIRIUSのHバンドのデータです。  
FITSファイルの名前は  **ir[4桁のフレーム番号].fits** となっています。  (SIRIUSのオリジナルのファイル名から変更しています)

| フレーム番号| オブジェクト | 積分時間   (1枚あたり) |  
| :-: | :- | :- |
|0001-0005 |  dome_flat_off  | 5秒 |
|0006-0010 | dome_flat_on  | 5秒|
|0011-0019 | target1  | 5秒 |   
|0020-0028 | target2  | 10秒  | 
|0029-0033 | dark  | 5秒  |
|0034-0038 | dark  | 10秒 |

フラット補正用のデータを、ドームフラットをランプをONとOFFでそれぞれ5枚1セットで取得しています。  
天体については2天体をそれぞれディザリングして9枚づつ取得しています。  
最後にダークフレームを5秒と10秒をそれぞれ5枚づつ取得しています。(ダークフレームは、天体の観測に使用した積分時間に対応するものが必要です。)


### target1の1次処理  

ここでは、target1を対象として、1次処理を行ってみます。  


#### ダーク補正用フレームの作成

「生データからダークを引き」のダークは、通常、複数枚(10枚程度)のダークの平均(メジアン)から作成するダーク補正用フレームを指します。  
平均(メジアン)をとることで、もっともらしいダーク補正用フレームを作成します。S/Nの良いフレームとなり、平均(メジアン)をとることで、偶然読み出しが変なフレームが混ざっていても、その影響を無しに、あるいは小さくすることができます。  

#### iraf.imcombine 

irafのimcombineがまさに、複数枚のフレームの平均を作成するためのタスクです。  


In [1]:
from pyraf import iraf

In [20]:
iraf.imcombine.unlearn()

ファイル名をカンマ区切りで並べた文字列をinputとして与えます。するとこれらのファイルをスタックして平均を計算してくれます。

In [26]:
iraf.imcombine.input = 'sample_data2/ir0029.fits, sample_data2/ir0030.fits, ' + \  # 長いので折り返し
    'sample_data2/ir0031.fits, sample_data2/ir0032.fits, sample_data2/ir0033.fits'

出力先ファイル名をdark5.fitsとし、medianで平均をとります。

In [27]:
iraf.imcombine(output='dark5.fits', combine='median')  


Jul 21 18:16: IMCOMBINE
  combine = median, scale = none, zero = none, weight = none
  blank = 0.
                Images 
  sample_data2/ir0029.fits
  sample_data2/ir0030.fits
  sample_data2/ir0031.fits
  sample_data2/ir0032.fits
  sample_data2/ir0033.fits

  Output image = dark5.fits, ncombine = 5


### おまけ  
'sample_data2/ir0029.fits, sample_data2/ir0030.fits, sample_data2/ir0031.fits, sample_data2/ir0032.fits, sample_data2/ir0033.fits' って長いですよね。ここでは5つのファイルだけでしたが、もっと長くなることだってありえます。  
下のように、globを利用して、こんなふうに該当する文字列を作成することができます。

In [18]:
comstr = '' 
for item in glob.glob('sample_data2/*.fits'):
    if 29 <= int(item[-7:-5])  <= 33:  #  4桁の番号の最後の二桁
        comstr += item + ',' 

In [30]:
comstr[:-1]  # 末尾の , を削除

'sample_data2/ir0029.fits,sample_data2/ir0030.fits,sample_data2/ir0031.fits,sample_data2/ir0032.fits,sample_data2/ir0033.fits'

In [28]:
#iraf.help('imcombine')